In [2]:
.libPaths("/home/senyaaa/BI-PRS/packages")
#.libPaths("/home/senyaaa/BI-PRS/packages") # change to ur folder
#.libPaths("/home/senyaaa/BI-PRS/packages")

library(eurostat)

In [3]:
surname <- "pogodin"
L <- nchar(surname)
K <- 4
M <- ((K + L) * 47) %% 11 + 1
print(M)

[1] 1


In [4]:
srch <- search_eurostat(pattern='Police-recorded offences by NUTS 3 regions')
srch

title,code,type,last.update.of.data,last.table.structure.change,data.start,data.end,values,hierarchy
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>
Police-recorded offences by NUTS 3 regions,crim_gen_reg,dataset,05.12.2023,05.12.2023,2008,2021,216947,4
Police-recorded offences by NUTS 3 regions,crim_gen_reg,dataset,05.12.2023,05.12.2023,2008,2021,216947,4


In [19]:
nuts3_cz <- c(
  "CZ010", "CZ020", "CZ031", "CZ032", "CZ041",
  "CZ042", "CZ051", "CZ052", "CZ053", "CZ063",
  "CZ064", "CZ071", "CZ072", "CZ080"
)

data <- get_eurostat(id = "crim_gen_reg")
czech_data <- data[data$geo==nuts3_cz,]
#czech_data <- data[data$TIME_PERIOD=='2021-01-01',]
czech_data_2021 <- czech_data[czech_data$TIME_PERIOD=='2021-01-01',]
czech_data_2021

Dataset query already saved in cache_list.json...

Reading cache file /tmp/Rtmp6eDsq1/eurostat/adb2a8f5bb5815758c4af1b2ad610318.rds

Table  crim_gen_reg  read from cache file:  /tmp/Rtmp6eDsq1/eurostat/adb2a8f5bb5815758c4af1b2ad610318.rds

Warning message in data$geo == nuts3_cz:
“longer object length is not a multiple of shorter object length”


freq,unit,iccs,geo,TIME_PERIOD,values
<chr>,<chr>,<chr>,<chr>,<date>,<dbl>
A,NR,ICCS0101,CZ051,2021-01-01,3.00
A,NR,ICCS0401,CZ031,2021-01-01,43.00
A,NR,ICCS0501,CZ071,2021-01-01,893.00
A,NR,ICCS05012,CZ010,2021-01-01,1105.00
A,NR,ICCS05012,CZ032,2021-01-01,359.00
A,NR,ICCS05012,CZ051,2021-01-01,216.00
A,NR,ICCS0502,CZ042,2021-01-01,2386.00
A,NR,ICCS050211,CZ041,2021-01-01,54.00
A,NR,ICCS050211,CZ053,2021-01-01,59.00
